In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T
import torchutils as tu
import json

import numpy as np
import torchutils as tu
import matplotlib.pyplot as plt
import time

In [2]:
trnsfrms = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(), # три функции для аугментации
    T.RandomRotation(45),
    T.ColorJitter(brightness=.5, hue=.3),
    T.ToTensor()
    ]
)

In [3]:
import torchvision
train_dataset = torchvision.datasets.ImageFolder('Agricultural-crops/train',
                                                 transform=trnsfrms)
valid_dataset = torchvision.datasets.ImageFolder('Agricultural-crops/valid',
                                                 transform=trnsfrms)

In [4]:
idx2class= {j: i for i, j in train_dataset.class_to_idx.items()}

In [6]:
batch_size = 32

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_dataset, shuffle=True, batch_size=batch_size)

In [7]:
from torchvision.models import resnet50, ResNet50_Weights

model = resnet50(weights=ResNet50_Weights.DEFAULT)
model

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /Users/lis/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:07<00:00, 13.8MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(2048, 30)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
device = 'mps'
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
fake_batch = torch.randn(batch_size, 3, 224, 224, device=device)
tu.get_model_summary(model.to(device), fake_batch)

Layer                                         Kernel               Output          Params             FLOPs
0_conv1                                      [3, 64, 7, 7]   [32, 64, 112, 112]       9,408   3,776,446,464
1_bn1                                                 [64]   [32, 64, 112, 112]         128     102,760,448
2_relu                                                   -   [32, 64, 112, 112]           0               0
3_maxpool                                                -     [32, 64, 56, 56]           0               0
4_layer1.0.Conv2d_conv1                     [64, 64, 1, 1]     [32, 64, 56, 56]       4,096     411,041,792
5_layer1.0.BatchNorm2d_bn1                            [64]     [32, 64, 56, 56]         128      25,690,112
6_layer1.0.ReLU_relu                                     -     [32, 64, 56, 56]           0               0
7_layer1.0.Conv2d_conv2                     [64, 64, 3, 3]     [32, 64, 56, 56]      36,864   3,699,376,128
8_layer1.0.BatchNorm2d_bn2  